In [1]:
class Decoder:
    '''
    Processes the next bit
    '''
    def process_next_bit(self, bit):
        pass

    '''
    Outputs the final data as a string
    '''
    def finalize(self) -> str:
        pass

In [2]:
from enum import IntEnum

In [3]:
class SWDecoder(Decoder):
    # ------------------------------------------------------
    # Letter State Machine
    # ------------------------------------------------------
    def __init__(self, letters = "ETIANMSURWDKGOHVF L PJBXCYZQ"):
        self.letters = letters
        self.current_letter = 0
        self.beat_error_range = 0.15

        self.beat_duration = 1 # How many bits is a beat?
        self.prev_bit = 0 # The value of the previous bit
        self.num_of_bit = 0 # The number of occurrences of "prev bit"

    '''
    Initializes the machine
    '''
    def init_letters(self):
        self.current_letter = 0

    '''
    Shifts the state machine to the next letter

    Params:
    - is_dash(bool): is really some bit indiciating if the incoming symbol is a dot or dash
    '''
    def shift_letter(self, is_dash: bool) -> None:
        self.current_letter = self.current_letter * 2 + is_dash + 1

    '''
    Gets the letter that the current state machine is on

    Returns:
    The length 1 string that has the character
    None, if CURRENT_LETTER is not in the range of the list of letters
    '''
    def get_letter(self) -> str: 
        ret_letter = ""

        if self.current_letter > 0 and self.current_letter <= len(self.letters):
            ret_letter = self.letters[self.current_letter - 1]

        self.current_letter = 0

        return ret_letter

    '''
    Finishes the machine and grabs the last letter
    '''
    def finalize(self) -> str:
        return self.get_letter()
    

    '''
    A list of the possible parsings from the morse code
    '''
    class MEANING(IntEnum):
        # Symbol
        DOT = 0
        DASH = 1

        # Pauses
        NEXT_SYMBOL = 2
        NEXT_LETTER = 3
        NEXT_WORD = 4

        # Unknown
        UNKNOWN = -1

    '''
    Removes noise from the beat_duration

    Params:
    - beat_nums(float): the rough number of beats
    '''
    def remove_noise(self, beat_nums: float):
        error_ranges = [(dur, dur * (1 - self.beat_error_range), dur * (1 + (self.beat_error_range if dur != 7 else 1))) for dur in (1,3,7)]

        for (duration, duration_min, duration_max) in error_ranges:
            if beat_nums >= duration_min and beat_nums <= duration_max:
                return duration

        return -1
    
    '''
    Processes some parsed information and interacts with the "letter" state machine

    Params:
    - meaning (MEANING): some parsed info from the bitstream input

    Returns:
    Either a letter, a letter + a space, or None
    '''
    def process(self, meaning: MEANING) -> str | None:
        ret_proc = None
        match meaning:
            case SWDecoder.MEANING.DOT:
                self.shift_letter(SWDecoder.MEANING.DOT)
            case SWDecoder.MEANING.DASH:
                self.shift_letter(SWDecoder.MEANING.DASH)
            case SWDecoder.MEANING.NEXT_LETTER:
                ret_proc = self.get_letter()
            case SWDecoder.MEANING.NEXT_WORD:
                ret_proc = self.get_letter() + " "

        return ret_proc
    
    # ------------------------------------------------------
    # Beat Decoder
    # ------------------------------------------------------
    '''
    Determines what the last string of bit B means

    Returns:
    The information as one of the discrete meanings above.
    '''
    def parse_prev_inputs(self):
        beats = self.remove_noise(self.num_of_bit / self.beat_duration)

        meaning = None
        match (self.prev_bit, beats):
            case (1, 1):
                meaning = SWDecoder.MEANING.DOT
            case (1, 3):
                meaning = SWDecoder.MEANING.DASH
            case (0, 1):
                meaning = SWDecoder.MEANING.NEXT_SYMBOL
            case (0, 3):
                meaning = SWDecoder.MEANING.NEXT_LETTER
            case (0, 7):
                meaning = SWDecoder.MEANING.NEXT_WORD
            case _:
                meaning = SWDecoder.MEANING.UNKNOWN
        return meaning

    '''
    Processes the next bit

    Params:
    - bit(int): either a 0 or 1 that is either an "on" or "off" signal from the sender

    Returns:
    A letter if we have reached the end of a letter/word, else None
    '''
    def process_next_bit(self, bit: int) -> str | None:
        if bit == self.prev_bit:
            self.num_of_bit += 1
        else:
            meaning = self.parse_prev_inputs()
            maybe_output = self.process(meaning)

            self.prev_bit = bit
            self.num_of_bit = 1

            return maybe_output

In [4]:
import sys

In [6]:
from pynq import Overlay, DefaultIP, allocate
import numpy as np

In [7]:
# class Decoder(DefaultIP):
#     CONTROL_REGISTER = 0x0
#     def __init__(self, description):
#         super().__init__(description = description)
        
#     bindto = ['xilinx.com:hls:processNextBit:1.0']
        
#     def start(self):
#         self.write(Decoder.CONTROL_REGISTER, 0x01);

In [8]:
class HWDecoder(Decoder): # "/home/xilinx/pynq/overlays/project/decoder/process_next_bit.bit"
    def __init__(self, overlay_file, stream_buf_size = 100) -> None:
        self.overlay = Overlay(overlay_file)
        self.send_channel = self.overlay.processNextBit.axi_dma.sendchannel
        self.recv_channel = self.overlay.processNextBit.axi_dma.recvchannel

        self.stream_buf_size = stream_buf_size
        self.bit_list = []

    def process(self, in_bits, recv_chars):
        # decoder.start()
        self.send_channel.start()
        self.recv_channel.start()
        self.send_channel.transfer(in_bits)
        self.send_channel.wait()
        self.recv_channel.transfer(recv_chars)
        self.recv_channel.wait()
        
        return recv_chars
    
    def process_next_bit(self, bit, finalize = False):
        if finalize or len(self.bit_list) == self.stream_buf_size:
            list_capacity = len(self.bit_list)

            in_bits = allocate(shape=(list_capacity,), dtype=np.uint32)
            recv_chars = allocate(shape=(list_capacity,), dtype=np.uint32)
            
            in_bits[:] = self.bit_list[:]
            print("in_bits", in_bits)
            output = self.process(in_bits, recv_chars)
            print("output", output)
            
            a_offset = ord('a')
            letter_list = [(chr(letter+a_offset-1) if letter else ' ') for letter in recv_chars]
            return_letter = ''.join(letter_list)
            self.bit_list = []
            del in_bits, recv_chars
            
            return return_letter
        else:
            self.bit_list.append(bit)

    def finalize_hw(self):
        return self.process_next_bit(0, finalize = True)

In [34]:
from pydub import AudioSegment
import numpy as np
import scipy.signal as signal
import math

In [42]:
class FFT:
    def __init__(self, signal_data: list, sample_rate: int) -> None:
        self.signal_data = signal_data
        self.sample_rate = sample_rate

    def transform(self) -> tuple[float, float, float]:
        fft_result = np.fft.fft(self.signal_data, self.sample_rate)
        freqs = np.fft.fftfreq(len(self.signal_data), 1/self.sample_rate)

        max_freq_index = np.argmax(fft_result)
        self.max_freq = freqs[max_freq_index]  

        theshold = 0.1 * self.max_freq

        lower_bound = np.where(fft_result[:max_freq_index] < theshold)[0][-1]
        upper_bound = np.where(fft_result[max_freq_index+1:] > theshold)[0][0] + max_freq_index + 1

        # print(lower_bound, upper_bound)

        self.bandwidth = freqs[upper_bound] - freqs[lower_bound]
        
        # print(freqs[lower_bound], freqs[upper_bound])
        
        return self.sample_rate, self.max_freq, self.bandwidth


In [43]:
class Filter:
    def filter(self, unfiltered: list, sample_rate: int):
        pass

In [63]:
class SWAudioFilter:
    def __init__(self, fft: FFT = FFT) -> None:
        self.fft = fft

    def gen_coef(self, num_coef: int, sample_rate: int, max_freq: float, bandwidth: float) -> list:
        lower_cutoff = max_freq - bandwidth / 2
        upper_cutoff = max_freq + bandwidth / 2

        coefficients = list()

        for i in range(int(num_coef)):
            t = (i - (num_coef - 1) / 2) / sample_rate

            next_coef = None
            if i == (num_coef - 1) / 2:
                next_coef = 2 * math.pi * bandwidth
            else:
                next_coef = (math.sin(2 * math.pi * upper_cutoff * t) 
                             - math.sin(2 * math.pi * lower_cutoff * t)) / (math.pi * t)
                
            coefficients.append(next_coef)

        return coefficients


    def filter(self, unfiltered: list, sample_rate: int):
        fft = self.fft(unfiltered, sample_rate)
        sampling_rate, max_freq, bandwidth = fft.transform()
    
        print(sampling_rate, max_freq, bandwidth)

        # Bandwidth/ sampling_rate = num_of_coefs?
        # Bandwidth is pretty specific, so maybe keep it at a hard like 11?
        num_of_coefs = 200
        coefficients = self.gen_coef(num_of_coefs, sampling_rate, max_freq, bandwidth)

        print(coefficients)

        shift_reg = [0] * num_of_coefs
        filtered = list()
        accumulator = 0
        data = 0

        # TODO: finish filter
        for unfil_data in unfiltered:
            for i in range(num_of_coefs - 1, 0, -1):
                if i == 0:
                    shift_reg[0] = unfil_data
                    data = unfil_data
                else:
                    shift_reg[i] = shift_reg[i-1]
                    data = shift_reg[i]

                accumulator += data * coefficients[i]

            filtered.append(accumulator)

        return np.array(filtered)

In [64]:
audio = AudioSegment.from_file('SOS.mp3')
sample_rate = audio.frame_rate
signal_data = np.array(audio.get_array_of_samples())
filter = SWAudioFilter(FFT)
filtered = filter.filter(signal_data, sample_rate)

from scipy.io.wavfile import write
write('filtered_sos.wav', sample_rate, filtered)

44100 29.08092380798969 0.2774897309922686
[0.5084813494448011, 0.5093983419718408, 0.5103065895355428, 0.511206076543311, 0.5120967875529419, 0.5129787072728806, 0.5138518205625183, 0.514716112432432, 0.5155715680446328, 0.51641817271283, 0.5172559119027041, 0.5180847712321023, 0.5189047364713435, 0.5197157935434478, 0.5205179285243343, 0.5213111276431274, 0.5220953772823331, 0.522870663978116, 0.5236369744205176, 0.5243942954536832, 0.5251426140760659, 0.525881917440705, 0.5266121928553565, 0.5273334277828164, 0.528045609841084, 0.5287487268035482, 0.5294427665992434, 0.5301277173130073, 0.5308035671857937, 0.5314703046147136, 0.5321279181533608, 0.5327763965119603, 0.5334157285575646, 0.5340459033142685, 0.5346669099633004, 0.5352787378434114, 0.5358813764508114, 0.5364748154395471, 0.5370590446215879, 0.5376340539669526, 0.5381998336040508, 0.5387563738196613, 0.5393036650592328, 0.5398416979269534, 0.5403704631859982, 0.5408899517586169, 0.5414001547263403, 0.5419010633301077, 0.5

In [5]:
# ------------------------------------------------------
# Translation
# ------------------------------------------------------
def translate(input_file, out = None, decoder_cls: Decoder = SWDecoder):
    if out is None:
        stdout = sys.stdout
    else:
        stdout = open(out, "w")

    decoder = decoder_cls()

    with open(input_file, "r") as in_file:
        decoder.init_letters()
        for line in in_file:
            for bit in line:
                if bit in [" ", "\n"]:
                    continue

                bit = int(bit)
                
                letter = decoder.process_next_bit(bit)

                if letter is not None:
                    print(letter, end = "", file = stdout)
                    
        print(decoder.finalize(), file = stdout)

    if out is not None:
        stdout.close()

In [6]:
from pathlib import Path
import time

In [7]:
'''
Basic Test Template to make other tests out of
'''
def translate_test(in_file, golden_file, result_file = None, time_it = False, **kwargs):
    if result_file is None:
        dot_split = in_file.split(".")
        result_file = "{}_out.{}".format("".join(dot_split[:-1]), dot_split[-1])
    
    timing = time.time()

    translate(in_file, out=result_file, **kwargs)

    if time_it:
        timing = time.time() - timing
    else:
        timing = 0

    correct_output = True

    with open(result_file, "r") as results:
        with open(golden_file, "r") as golden:
            next_result = results.readline().strip().lower()
            next_golden = golden.readline().strip().lower()

            while next_result != "" and next_golden != "":
                if next_result != next_golden:
                    print(f"'{next_result}' does not match '{next_golden}'")
                    correct_output = False
                    break
                next_result = results.readline()
                next_golden = golden.readline()

            if next_result != next_golden:
                print(f"One was empty:\n'{next_result}' does not match '{next_golden}'")
                correct_output = False
            
#     Path(result_file).unlink()

    if correct_output:
        timing_statement = "" if not time_it else f" in {timing:4.2} seconds"
        print(f"Successfully translated {in_file}{timing_statement}")

    return correct_output
                 


In [8]:
'''
Tests if the alphabet with no spaces is accurate
'''
def test1():
    translate_test("decoder_tests/test1.txt", "decoder_tests/golden_out1.txt", time_it = True)
    
test1()

Successfully translated decoder_tests/test1.txt in 0.0046 seconds


In [9]:
'''
Tests if the alphabet with 1 space is accuate
'''
def test2():
    translate_test("decoder_tests/test2.txt", "decoder_tests/golden_out2_3.txt", time_it = True)

test2()

Successfully translated decoder_tests/test2.txt in 0.002 seconds


In [10]:
'''
Tests if multiple spaces are ommitted
'''
def test3():
    translate_test("decoder_tests/test3.txt", "decoder_tests/golden_out2_3.txt", time_it = True)

test3()

Successfully translated decoder_tests/test3.txt in 0.002 seconds


In [11]:
'''
Tests the "the quick brown fox jumps over the lazy dog"
'''
def test4():
    translate_test("decoder_tests/test4.txt", "decoder_tests/golden_out4.txt", time_it = True)

test4()

Successfully translated decoder_tests/test4.txt in 0.0057 seconds


In [79]:
def testA_hw():
    translate_test("decoder_tests/untitled.txt", "decoder_tests/untitled1.txt", time_it = True, decoder_cls = HWDecoder)
testA_hw()

[1, 0, 1, 1, 1, 0, 0, 0]
in_bits [1 0 1 1 1 0 0 0]
output [0 0 0 0 0 0 0 0]
'        ' does not match 'a'
One was empty:
'        ' does not match 'a'


In [80]:
def test1_hw():
    translate_test("decoder_tests/test1.txt", "decoder_tests/golden_out1.txt", time_it = True, decoder_cls = HWDecoder)
    
test1_hw()

[1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1]
in_bits [1 0 1 1 1 0 0 0 1 1 1 0 1 0 1 0 1 0 0 0 1 1 1 0 1 0 1 1 1 0 1 0 0 0 1 1 1
 0 1 0 1 0 0 0 1 0 0 0 1 0 1 0 1 1 1 0 1 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 1 0
 1 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 1 1 1 0 1 1 1 0 1 1]
output [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1,